In [1]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents. 

In [2]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)



### Generate binary DES with lower mole fractions of 0.2, 0.3 (QAS, HBD) and upper mole fractions of 0.7, 0.8 (QAS, HBD). Produce 96 samples with 192 trials for the clustering.

In [3]:
Binary_DES_Generator(0.2, 0.3, 0.7, 0.8, 96, 192)

 3840 feasible DES samples generated, clustered into 96 samples


array([[0.54564805, 0.45435195],
       [0.38321382, 0.61678618],
       [0.32434025, 0.67565975],
       [0.46212625, 0.53787375],
       [0.64766886, 0.35233114],
       [0.26866796, 0.73133204],
       [0.43098344, 0.56901656],
       [0.59129215, 0.40870785],
       [0.49927881, 0.50072119],
       [0.39942272, 0.60057728],
       [0.22948862, 0.77051138],
       [0.3473242 , 0.6526758 ],
       [0.52285076, 0.47714924],
       [0.29278777, 0.70721223],
       [0.56428959, 0.43571041],
       [0.61684063, 0.38315937],
       [0.68904484, 0.31095516],
       [0.44618944, 0.55381056],
       [0.48090995, 0.51909005],
       [0.25191635, 0.74808365],
       [0.41656176, 0.58343824],
       [0.5315265 , 0.4684735 ],
       [0.61122992, 0.38877008],
       [0.3661846 , 0.6338154 ],
       [0.5092443 , 0.4907557 ],
       [0.67741139, 0.32258861],
       [0.31000501, 0.68999499],
       [0.58000106, 0.41999894],
       [0.47316734, 0.52683266],
       [0.40758957, 0.59241043],
       [0.

### May also save the output as a variable. Note you will get different mole fractions each time so be sure to keep track if you need them.

In [4]:
A = Binary_DES_Generator(0.3, 0.3, 0.7, 0.8, 96, 192)
A 

 3812 feasible DES samples generated, clustered into 96 samples


array([[0.39263888, 0.60736112],
       [0.54493406, 0.45506594],
       [0.47598803, 0.52401197],
       [0.62747868, 0.37252132],
       [0.31663748, 0.68336252],
       [0.43347427, 0.56652573],
       [0.58511062, 0.41488938],
       [0.5079548 , 0.4920452 ],
       [0.35702472, 0.64297528],
       [0.41160259, 0.58839741],
       [0.68786183, 0.31213817],
       [0.45325085, 0.54674915],
       [0.56631071, 0.43368929],
       [0.37558748, 0.62441252],
       [0.64660272, 0.35339728],
       [0.52560332, 0.47439668],
       [0.33984698, 0.66015302],
       [0.48775018, 0.51224982],
       [0.6035977 , 0.3964023 ],
       [0.49430749, 0.50569251],
       [0.46565877, 0.53434123],
       [0.66908474, 0.33091526],
       [0.42204009, 0.57795991],
       [0.44004988, 0.55995012],
       [0.55629304, 0.44370696],
       [0.51535183, 0.48464817],
       [0.32656066, 0.67343934],
       [0.37142295, 0.62857705],
       [0.5772832 , 0.4227168 ],
       [0.61736515, 0.38263485],
       [0.

### The next function will combine the previous function with code to convert mole fractions to a list of lists of volumes that can be directly inputed into code for sample preparation in a pipetting robot (OT-2).

In [5]:
def convert_mole_fractions_to_volumes(stock_QAS, stock_HBD, min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    DES_mole_fractions = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 125  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [6]:
convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

 3840 feasible DES samples generated, clustered into 96 samples


/Users/studentuser/miniconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[[81.09038498580293,
  56.923142445954866,
  96.67801736686815,
  69.35974870892383,
  87.94989373300388,
  74.22028312553101,
  62.709407854627166,
  92.32323640604353,
  103.76478998642827,
  76.78983245484045,
  49.790385089204655,
  84.10350803868509,
  65.57396784825173,
  98.45175930148359,
  79.91103476659327,
  71.14583512648073,
  59.58217145515696,
  90.99160691511196,
  53.701419367439954,
  85.73545370806072,
  67.62283094269476,
  72.98157351793273,
  94.50742838521788,
  100.83414732158874,
  82.24368915681592,
  78.38065961440068,
  58.353591222532586,
  75.18564212394374,
  64.28515507898221,
  88.86500090902476,
  46.67215819509109,
  86.80007454435807,
  60.84845710567913,
  93.47709308660178,
  55.46398926359825,
  83.13685601630056,
  77.32191558091235,
  89.92695573515076,
  71.64810489214213,
  66.61805713473036,
  97.77988201448315,
  52.1468811772669,
  84.5932164889453,
  99.61110140968383,
  95.24060949827512,
  68.2089111259583,
  78.86680468225754,
  48.2726

### Alternatively, you could use this next function if you already have a list you want to convert (as before when we saved the output of the des mole fractions)

In [7]:
def convert_mole_fractions_to_volumes_2(DES_mole_fractions, stock_QAS, stock_HBD):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 150  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [8]:
convert_mole_fractions_to_volumes_2(A, 3, 2)

[[45.17658162576325,
  66.58893132011755,
  56.5751714499863,
  79.34327119411776,
  35.40007916670883,
  50.66860583701284,
  72.68787826483543,
  61.14914799631755,
  40.52531935358732,
  47.70549415894411,
  89.2500933015373,
  53.391695958730956,
  69.80891698889378,
  42.93389276693793,
  82.4258684698594,
  63.72502727160363,
  38.326402533598056,
  58.24462771455041,
  75.56298381560714,
  59.18214091322101,
  55.12187135928526,
  86.11442278173078,
  49.11326456305567,
  51.56935076607354,
  68.29293084536594,
  62.22432284828651,
  36.64500508284595,
  42.39057266539402,
  71.48378226255025,
  77.73307931278438,
  46.49641869905226,
  74.17083043835224,
  34.29201980409897,
  65.19432865854017,
  44.23426518382675,
  81.12650624794232,
  57.15340009877217,
  83.5866697439397,
  59.779162588792474,
  41.733436176120534,
  54.69776215691965,
  37.818619584074824,
  87.59328960354065,
  39.52180193676397,
  53.84003758530906,
  76.41666723767386,
  69.2706512281538,
  52.96961299

In [9]:
from opentrons import labware, instruments, robot
from sqlite3 import IntegrityError
robot.reset()
#####################################################################################################################################################
#####################################################################################################################################################		

# Import Labware

tiprack_300 = labware.load("opentrons-tiprack-300ul", '1')      
tiprack_1000 = labware.load("tiprack-1000ul", '4')                
Stock1 = labware.load("opentrons-tuberack-15ml", '2')                                           
wellplate_96 = labware.load("96-flat", '3')
trash = robot.fixed_trash

#####################################################################################################################################################
#####################################################################################################################################################

# Import Pipettes

P1000 = instruments.P1000_Single(
    mount = 'right',
    tip_racks = [tiprack_1000],
    trash_container = trash
)

P300 = instruments.P300_Single(
    mount='left',
    tip_racks=[tiprack_300],
    trash_container=trash 
)

#####################################################################################################################################################
#####################################################################################################################################################

#Insert DES Generator here. 

reagents1 = convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

reagent_pos1 = ['A1', 'A3']



#####################################################################################################################################################
#####################################################################################################################################################
# STOCK LABWARE 1

robot.home()                                                    # Homes robot and prevents any pipette bugs 
for counter, reagent in enumerate(reagents1,0):
                                                                # These objects are temporary and will only exist within this loop
    source      = reagent_pos1[counter]                         # Counter is use to index an independent list (e.g. reagent_pos)
    P1000list   = [source]                                      # This is then added to both list - used in testing 				
    P300listn   = [source]


    P300.pick_up_tip()                                          # Picks up pipette tip for both P10 and P300 to allow to alternate
    P1000.pick_up_tip()
    for well_counter, values in enumerate(reagent):             # Specifies the well position and the volume of reagent being 
        if values == float(0):                                  # If volume is 0, well is skipped  
            pass
        elif values < float(300):
            P300.distribute(                                     # If volume below 300, P300 used not p1000. Greater than 300 P1000 is used.
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),          # Prevents submerging tip in solution, not completely sterile, but beneficial
                blow_out=True,                                  # Removes excess from tip
                rate=1,                                         # How quick it aspirates/dispenses, lower (ie 0.5) if stock viscous
                new_tip='never')
            P300.touch_tip(wellplate_96(well_counter))         # Touches tip to remove any droplets
            P300.blow_out(wellplate_96(well_counter))
        else: 
            P1000.distribute(
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),
                blow_out=True,
                rate=1,
                new_tip='never')
            P1000.touch_tip(wellplate_96(well_counter))
            P1000.blow_out(wellplate_96(well_counter))
    P1000.drop_tip()
    P300.drop_tip()

for c in robot.commands():
     print(c)


ModuleNotFoundError: No module named 'opentrons'